In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from collections import Counter

In [2]:
#This functione sorts the dataframe column and then add sequece number to dataframe
def give_sort_number(df,column_name,ascending_order=True):
    df = df.sort_values(by=[column_name], ascending=ascending_order).reset_index()
    df.index.name = "Sort_"+column_name
    df = df.reset_index()
    df["Sort_"+column_name] += 1
    return df

In [3]:
mainDF = pd.read_excel("Data//October_10//LabHoursData.xlsx")
mainDF.head(2)

,#,Machine Type,Machine,Wing,Floor,Run No,Pcs,StartDate,StartTime,StopDate,...,RunHours,IniWeight,FinalWeight,GrownWeight,Width,Length,Dmtr,Unit,Month,GrRate
0,NaN,Old,CA2,NaN,A,CA2-112,13,2022-10-01 02:10:00,2022-10-01 02:10:00,2022-10-08 14:10:00,...,180.0,46.95,144.82,97.87,11.318461,13.861538,50.71,C,2022-10 (Oct),13.67
1,NaN,Old,CA2,NaN,A,CA2-113,9,2022-10-08 20:30:00,2022-10-08 20:30:00,2022-10-15 12:30:00,...,160.0,69.27,146.18,76.91,12.832222,14.472222,46.92,C,2022-10 (Oct),13.56


In [4]:
final_DF = mainDF[["Machine Type", "Machine", "GrownWeight", "BreakdownTime", "GrowingHours", "Dmtr", "Stop Reasons", "LateStartReason"]]
#final_DF[final_DF.isna().any(axis=1)]
final_DF = final_DF[:4802]
final_DF = final_DF.rename(columns = {'Machine Type':'Machine_Type', 'Stop Reasons':'Stop_Reasons'})

final_DF.tail()

,Machine_Type,Machine,GrownWeight,BreakdownTime,GrowingHours,Dmtr,Stop_Reasons,LateStartReason
4797,New,BBA9,8.42820,11.17,21.65,42.22,"W LINE , TIDE O",ON TIME
4798,Old,BBE40,21.40920,0.00,52.75,43.66,"TIDE O , N/A",ON TIME
4799,New,AG27,121.92000,12.17,278,46.86,"TIDE O , N/A",HLD CLN
4800,New,AG27,60.52500,11.33,165,40.52,"HEAT , N/A",ON TIME
4801,New,AG27,11.93409,12.50,22.15,39.67,"V LEAK , TIDE O",HLD CLN


In [5]:
def count_words(df_temp,column_name):
    df_temp[column_name] = df_temp[column_name].str.rstrip('')
    df_temp[column_name] = df_temp[column_name].str.lstrip('')

    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(',,',',')

    df_temp[column_name] = df_temp[column_name].str.replace('\' ','\'')
    df_temp[column_name] = df_temp[column_name].str.replace(' \'','\'')

    df_temp[column_name] = df_temp[column_name].str.rstrip(',')
    df_temp[column_name] = df_temp[column_name].str.lstrip(',')

    df_temp[column_name] = df_temp[column_name].str.split(',')
    df_temp[column_name] = df_temp[column_name].apply(lambda x: Counter(x))
    
    df_temp[column_name] = df_temp[column_name].apply(lambda x: dict(x))
    
    return df_temp

In [6]:
#join all string values of Stop_Reasons in groupby
df_Stop_reason = final_DF.groupby(['Machine','Machine_Type'])['Stop_Reasons'].agg(lambda x: ','.join(x.dropna())).reset_index()

#join all string values of LateStartReason in groupby
df_LateStartReason = final_DF.groupby(['Machine','Machine_Type'])['LateStartReason'].agg(lambda x: ','.join(x.dropna())).reset_index()

In [7]:
df_Stop_reason['Stop_Reasons_counts'] = df_Stop_reason['Stop_Reasons']
df_Stop_reason = count_words(df_Stop_reason,'Stop_Reasons_counts')

df_LateStartReason['LateStartReason_counts'] = df_LateStartReason['LateStartReason']
df_LateStartReason = count_words(df_LateStartReason,'LateStartReason_counts')

In [8]:
#======================================Dmtr sum===============================================

# df_machine_less35 = pd.merge(
#     final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','BreakdownTime', 'Dmtr'].sum().reset_index(), 
#     final_DF.groupby(['Machine', 'Machine_Type'])['GrowingHours'].sum().reset_index(), 
#     how="left", on=["Machine", "Machine_Type"])

#======================================Dmtr average===============================================

df_machine_less35 = pd.merge(pd.merge(
                            final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','BreakdownTime'].sum().reset_index(),
                            final_DF.groupby(['Machine', 'Machine_Type'])['Dmtr'].mean().reset_index(), 
                                      how="left", on=["Machine", "Machine_Type"]),
                      final_DF.groupby(['Machine', 'Machine_Type'])['GrowingHours'].sum().reset_index(),
                             how="left", on=["Machine", "Machine_Type"])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7776\4146050627.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','BreakdownTime'].sum().reset_index(),


In [9]:
df_machine_less35

,Machine,Machine_Type,GrownWeight,BreakdownTime,Dmtr,GrowingHours
0,AA1,Old,264.030160,68.67,41.901667,675.31
1,AA10,Old,272.452150,68.33,44.670000,675.65
2,AA11,Old,264.602140,85.16,44.988333,658.81
3,AA12,Old,244.256410,58.09,42.404000,685.9
4,AA13,Old,265.286690,56.84,44.416000,687.14
...,...,...,...,...,...,...
923,CC16,Old,296.671520,30.33,44.294000,713.64
924,CC17,Old,328.814920,46.42,46.428571,697.57
925,CC20,Old,285.491085,47.17,42.951667,696.82
926,CC21,Old,363.617610,38.83,46.235000,705.16


In [10]:
df_machine_less35["GrownWeight_less_35"] = df_machine_less35['GrownWeight'] - (df_machine_less35["GrownWeight"] * 35 / 100)

In [11]:
df_machine_less35.index.name = "ID"
df_machine_less35.reset_index(inplace=True)
df_machine_less35["ID"] += 1

In [12]:
df_machine_less35.GrownWeight = pd.to_numeric(df_machine_less35.GrownWeight)
df_machine_less35 = df_machine_less35.sort_values(by=['GrownWeight'], ascending=False).reset_index()
df_machine_less35.index.name = "Sorted_GrownWeight"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["Sorted_GrownWeight"] += 1
df_machine_less35.index.name = "deleteA"

df_machine_less35.BreakdownTime = pd.to_numeric(df_machine_less35.BreakdownTime)
df_machine_less35 = df_machine_less35.sort_values(by=['BreakdownTime'], ascending=False).reset_index()
df_machine_less35.index.name = "Sorted_BreakdownTime"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["Sorted_BreakdownTime"] += 1
df_machine_less35.index.name = "deleteB"

df_machine_less35.Dmtr = pd.to_numeric(df_machine_less35.Dmtr)
df_machine_less35 = df_machine_less35.sort_values(by=['Dmtr'], ascending=False).reset_index()
df_machine_less35.index.name = "Sorted_Dmtr"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["Sorted_Dmtr"] += 1
df_machine_less35.index.name = "deleteC"

df_machine_less35.GrowingHours = pd.to_numeric(df_machine_less35.GrowingHours)
df_machine_less35 = df_machine_less35.sort_values(by=['GrowingHours'], ascending=False).reset_index()
df_machine_less35.index.name = "Sorted_GrowingHours"
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35["Sorted_GrowingHours"] += 1
df_machine_less35.index.name = "deleteD"

In [13]:
df_machine_less35['Total_of_Sorted_Number'] = df_machine_less35['Sorted_GrownWeight'] + df_machine_less35['Sorted_BreakdownTime'] + \
                                        df_machine_less35['Sorted_Dmtr'] + df_machine_less35['Sorted_GrowingHours']

In [14]:
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35 = df_machine_less35.drop(["deleteA","deleteB","deleteC","deleteD","index"], axis=1)

In [15]:
df_machine_less35 = df_machine_less35.set_index("ID")
df_machine_less35 = df_machine_less35.sort_index()

In [16]:
df_results = pd.merge(pd.merge(df_machine_less35,df_Stop_reason,how="left", on=["Machine", "Machine_Type"]),df_LateStartReason,how="left", on=["Machine", "Machine_Type"])

In [17]:
df_results = df_results[['Machine_Type','Machine','GrownWeight','GrownWeight_less_35','Sorted_GrownWeight',
                         'BreakdownTime','Sorted_BreakdownTime','Dmtr','Sorted_Dmtr','GrowingHours','Sorted_GrowingHours',
                         'Total_of_Sorted_Number','Stop_Reasons_counts', 'LateStartReason_counts']]

In [18]:
if os.path.exists("final.xlsx"):
    os.remove("final.xlsx")
    
df_results.to_excel("final.xlsx", header=1, index_label="ID")

In [19]:
m,t = 'AA64','Old'
final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)]

,Machine_Type,Machine,GrownWeight,BreakdownTime,GrowingHours,Dmtr,Stop_Reasons,LateStartReason
981,Old,AA64,38.40246,0.00,111.67,39.33,"TIDE O , N/A",ON TIME
982,Old,AA64,74.08000,7.83,185,46.88,"TIDE O , V LEAK",ON TIME
983,Old,AA64,60.94500,10.00,190,40.68,"TIDE O , N/A",ON TIME
984,Old,AA64,82.52000,22.83,209,41.94,"TIDE O , N/A",POWER PROBLEM


In [20]:
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrownWeight.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].BreakdownTime.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Dmtr.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Dmtr.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrowingHours.sum())


255.94745999999998
40.66
168.83
42.2075
695.6700000000001
